In [2]:
import torch
print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

GPU available: True
GPU name: Tesla P100-PCIE-16GB


In [3]:
from huggingface_hub import login
login()

In [4]:
!pip install bitsandbytes>=0.39.0
!pip install --upgrade accelerate transformers datasets peft trl
!pip install streamlit
!npm install -g localtunnel

  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached peft-0.15.2-py3-none-any.whl.metadata (13 kB)
  Using cached trl-0.17.0-py3-none-any.whl.metadata (12 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 115.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      S

In [5]:
!wget https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
!unzip -o transcripts.zip -d transcripts/

--2025-05-07 19:59:13--  https://github.com/CS639-Data-Management-for-Data-Science/s25/raw/main/p6/transcripts.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip [following]
--2025-05-07 19:59:13--  https://raw.githubusercontent.com/CS639-Data-Management-for-Data-Science/s25/main/p6/transcripts.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 290933 (284K) [application/zip]
Saving to: ‘transcripts.zip’

transcripts.zip     100%[===================>] 284.11K  --.-KB/s    in 0.02s   

2025-05-07 19:59:13 (12.8 MB/s) - ‘transcri

Q1.1: Load a 4-bit quantized Llama-3.2-1B-Instruct model and and its tokenizer.

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [7]:
model_id = "meta-llama/Llama-3.2-1B-Instruct"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [8]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    low_cpu_mem_usage=True
)
model.to(device)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

2025-05-07 20:00:15.271304: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746648015.449929      99 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746648015.499717      99 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

Q1.2: Test your quantized model with different prompts (text generation).

In [11]:
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [12]:
prompt = "What are some fun traditions or events that students at UW–Madison look forward to every year?\n\n"
response = generate_response(prompt, max_new_tokens=150)
print(response)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


What are some fun traditions or events that students at UW–Madison look forward to every year?

Here are some fun traditions or events that students at UW–Madison look forward to every year:

1. **Greek Week**: A week-long celebration of Greek life, with events like Greek dance performances, food festivals, and charity events.
2. **Homecoming**: A high school event that takes place before the start of the fall semester, featuring football games, concerts, and other activities.
3. **Columbus Day Parade**: A annual parade that takes place in the fall, featuring floats, marching bands, and other performances.
4. **UW-Madison Football**: A annual football game that takes place in the fall, with the university's football team competing against other schools.
5. **Wine and Cheese Tastings


In [13]:
prompt = "What are some of the places to visit in Madison, Wisconsin?\n\n"
response = generate_response(prompt, max_new_tokens=150)
print(response)

What are some of the places to visit in Madison, Wisconsin?

Madison is a beautiful city with a rich history and cultural attractions. Here are some of the places to visit in Madison, Wisconsin:

1. **University of Wisconsin-Madison**: The university is one of the largest in the country and offers a wide range of academic programs. Take a stroll through the beautiful campus and visit the iconic Mendel's Garden.
2. **State Street**: This historic street is lined with shops, restaurants, and bars. It's a great place to explore and grab a bite to eat.
3. **Boulevard Park**: A beautiful park with walking trails, a playground, and a lake. It's a great place to relax and enjoy the scenery.
4. **The Olbrich Botanical Gardens**:


In [14]:
prompt = "How can one improve their time management skills?\n\n"
response = generate_response(prompt, max_new_tokens=150)
print(response)

How can one improve their time management skills?

Here are some tips to improve your time management skills:

1.  **Set clear goals**: Start by identifying what you want to achieve in a specific timeframe. This will help you focus on what's truly important and make sure you're spending your time wisely.
2.  **Prioritize tasks**: Make a list of all your tasks and then prioritize them based on their importance and urgency. This will help you tackle the most critical tasks first.
3.  **Use a planner or calendar**: Write down all your tasks, appointments, and deadlines in a planner or calendar. This will help you stay organized and keep track of your time.
4.  **Break tasks into smaller chunks**: Large tasks can be overwhelming. Break them down into smaller,


Q1.3: Identify a prompt where the model fails and analyze the failure.

In [15]:
prompt = "If yesterday was two days before Monday, what day is today?\n\n"
response = generate_response(prompt, max_new_tokens=150)
print(response)

If yesterday was two days before Monday, what day is today?

## Step 1: Determine the number of days between yesterday and today.
Yesterday was two days before Monday, which means today is two days after Monday.

## Step 2: Calculate the total number of days between yesterday and today.
Since yesterday was two days before Monday and today is two days after Monday, the total number of days between yesterday and today is 2 + 2 = 4 days.

## Step 3: Determine the day of the week that is four days after Monday.
Starting from Monday, adding four days lands on Friday.

The final answer is: $\boxed{Friday}$


The model failed for the above prompt possibly due to a couple of reasons:

1) The model we are using is smaller and probably struggles with multi-step reasoning problems like the one above where you need to first deduce what day of the week yesterday was and then you need to figure out what today is.
2) The model also lacks access to external sources like a calendar or other tools to test out its reasoning logic.

Q1.4: Enhance model responses by providing additional context using chat templates.

In [16]:
def apply_chat_template(role, prompt, max_new_tokens=100):
    messages = [{"role": "system",
                "content": role},
                {"role": "user", "content": prompt}]
    inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
    outputs = model.generate(inputs, max_new_tokens=max_new_tokens, pad_token_id=tokenizer.eos_token_id)
    print("Role:", role)
    print("Prompt:", prompt)
    print("Generated Response:", tokenizer.decode(outputs[0], skip_special_tokens=True))

In [17]:
prompt = "What should I choose as my starter if my rival who I want to beat chose Bulbasaur as their starter?"
role = "You are a wise and encouraging Pokémon professor who gives thoughtful advice to new trainers. Recommend them one Pokemon"
apply_chat_template(role, prompt, max_new_tokens=300)

Role: You are a wise and encouraging Pokémon professor who gives thoughtful advice to new trainers. Recommend them one Pokemon
Prompt: What should I choose as my starter if my rival who I want to beat chose Bulbasaur as their starter?
Generated Response: system

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

You are a wise and encouraging Pokémon professor who gives thoughtful advice to new trainers. Recommend them one Pokemonuser

What should I choose as my starter if my rival who I want to beat chose Bulbasaur as their starter?assistant

What an interesting situation. Considering Bulbasaur is a Grass-type Pokémon, I'd recommend a starter that complements its strengths.

As a rival, I'm sure they have a strong understanding of the Pokémon world. In that case, I'd suggest a starter that offers a different set of strengths and weaknesses.

I recommend choosing Charmander as your starter. Charmander is a Fire-type Pokémon, which would provide a nice contrast to Bulbasaur'

Yes, the prompt got the model to respond in a manner befitting of a Pokemon professor helping out the trainer at the start of their journey. It stayed in character and gave relevant strategic advice.

Q2.1: Test the model before fine-tuning.

In [18]:
prompt = "Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?"
role = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."
apply_chat_template(role, prompt, max_new_tokens=300)

Role: You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.
Prompt: Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?
Generated Response: system

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?assistant

In CS 639 Data Management for Data Science, six projects are covered, including:

1. **Data Cleaning**: Focuses on handling missing values, data normalization, and data preprocessing. Tools used include Pandas for data manipulation, NumPy for numerical computations, and Matplotlib for visualization.
2. 

Q2.2 Fine-tune the model on course lecture transcripts with LoRA.

In [19]:
from datasets import Dataset

from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer
import os

In [20]:
transcript_dir = "transcripts/transcripts"
test_ratio = 0.1
train_texts = []
test_texts = []
texts = []

for file_name in os.listdir(transcript_dir):
    if file_name.endswith(".txt"):
        with open(os.path.join(transcript_dir, file_name), "r", encoding="utf-8") as f:
          lines = f.readlines()
          
          split_idx = int(len(lines) * test_ratio)
          test_lines = lines[:split_idx]
          train_lines = lines[split_idx:]
          train_texts.append("".join(train_lines))
          test_texts.append("".join(test_lines))

In [21]:
train_dataset = Dataset.from_dict({"text": train_texts})
test_dataset = Dataset.from_dict({"text": test_texts})

In [22]:
def tokenize_data(data):
    tokenized = tokenizer(
        data["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )
    # Set the labels to be the same as input_ids for causal language modeling
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_train = train_dataset.map(tokenize_data, batched=True)
tokenized_test = test_dataset.map(tokenize_data, batched=True)

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

Map:   0%|          | 0/23 [00:00<?, ? examples/s]

In [23]:
tokenized_train

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 23
})

In [24]:
tokenized_test

Dataset({
    features: ['text', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 23
})

In [25]:
lora_config = LoraConfig(
    r=8,
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "o_proj", "k_proj", "v_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
)

In [26]:
training_args = TrainingArguments(
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=1,
    logging_dir="./logs",
    output_dir="./results",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    report_to="none"
)

In [27]:
trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    args=training_args,
    peft_config=lora_config,
)

Truncating train dataset:   0%|          | 0/23 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/23 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [28]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.930200,2.968326
2,2.847600,2.902081
3,2.636100,2.850378
4,2.621300,2.831767
5,2.633900,2.826171
6,2.414400,2.829195
7,2.342600,2.834560
8,2.266600,2.843247


TrainOutput(global_step=50, training_loss=2.6122322988510134, metrics={'train_runtime': 184.5846, 'train_samples_per_second': 1.246, 'train_steps_per_second': 0.271, 'total_flos': 577309237051392.0, 'train_loss': 2.6122322988510134})

Q2.3: Test the model after fine-tuning.

In [30]:
prompt = "Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?"
role = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."
apply_chat_template(role, prompt, max_new_tokens=300)

Role: You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.
Prompt: Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?
Generated Response: system

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

Can you tell me what the six projects covered in CS 639 Data Management for Data Science are and what tools each one focuses on in less than a 100 words?assistant

In CS 639, the six projects cover the following topics and focus on specific tools. Here's a brief overview of each project:

1. **Data Cleaning**: Focuses on data preprocessing, handling missing values, and data normalization. Utilizes tools like Pandas for data manipulation, NumPy for numerical comput

There seems to be a very slight improvement in the quality of the response.
The initial response felt very generic and like something that was just picked up from the internet. 
After fine-tuning, the model does seem to pick up some context from the lecture transcripts since it managed to identify that data warehousing was one of the projects and that Snowflake was used in this project,something which wasn't picked up at all initially.
However, there is still a lot of scope for improvement and the response isn't particularly accurate still.

Q3.3: Compare Fine-Tuning vs RAG

In [31]:
queries = [
    "What are some of the topics covered in the course?",
    "What is the course code number for Data Management for Data Science?",
    "What NoSQL database was the main focus of project P2 in CS639?",
    "Can you explain window functions as taught in this course?",
    "What SQL query clauses did we learn about in this course?"
]

role = "You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions."

for query in queries:
    apply_chat_template(role, query, max_new_tokens=300)

Role: You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.
Prompt: What are some of the topics covered in the course?
Generated Response: system

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

You are an instructor of CS 639 Data Management for Data Science course at UW-Madison, and are currently answering student questions.user

What are some of the topics covered in the course?assistant

The CS 639 Data Management for Data Science course covers a wide range of topics related to data management. Here are some of the key topics that are typically covered in the course:

1. **Data Modeling**: Students learn about data modeling, which is the process of designing and structuring the database schema to support the data requirements of the application. They learn about entities, attributes, relationships, and data types.

2. **Database Design**: Students learn about the design of the database, i